# SEN163A - Fundamentals of Data Analytics
# Assignment 3 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 25-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
short description of the assignment

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import time
import pickle
import datetime

In [4]:

#READING MAIN PAGE
#Get file
source_link = "https://jdestefani.github.io/SEN163A-TabularRazorArchives/"
response = requests.get(source_link)

#Read main file
main_page = BeautifulSoup(response.text)

for year_link in main_page.find_all('a', href=True):
    #READING YEAR PAGE & SAVING PKL FILE FOR EACH YEAR
    
    #Get file name of pkl file
    year_filename = year_link.get('href')
    year_filename = year_filename.removeprefix('./')
    year_filename = year_filename.removesuffix('.html')
    year_filename = "data/" + year_filename + ".pkl"
    
    #Clear lists of data for each year
    Data_Tuple = []
    
    #Get file  
    year_link = urljoin(source_link, year_link.get('href')) 
    response = requests.get(year_link)
    
    #Read year file
    year_page = BeautifulSoup(response.text)
     
    for month_link in year_page.find_all('a', href=True):
        #Get file
        month_link = urljoin(year_link, month_link.get('href'))
        response = requests.get(month_link)
        
        #Read month file
        month_page = BeautifulSoup(response.text)
        i = 0
        
        start  = time.time()
        
        for article_link in month_page.find_all('a', href=True):        
            #Get file
            article_link = urljoin(month_link, article_link.get('href')) 
            response = requests.get(article_link)
        
            #Read article file
            article_page = BeautifulSoup(response.text)
                       
            for div_element in article_page.find_all('div'):
                if div_element.get('class') == ['author']:
                    article_author = div_element.get_text()
                if div_element.get('class') == ['date']:
                    article_date = div_element.get_text()
                if div_element.get('class') == ['time']:
                    article_time = div_element.get_text()

            #Add data
            Data_Tuple.append((article_author, article_date, article_time))
            i += 1 
            #if i >= 10:
            #    break
        
        #Print after each month
        dur = round(time.time() - start,2)
        print('Read: ' + month_link)
        print('Added: ' + str(i) + ' articles')
        print('Duration: ' + str(dur) + ' seconds')
        
        #break

    #Save file after each year
    print()
    print("---")
    print("Saving: " + year_filename)
    df = pd.DataFrame(Data_Tuple)
    df.to_pickle(year_filename)
    print("Saved: " + str(len(df[0])) + " articles")
    print("---")
    print()
    
    #break
    

Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-1.html
Added: 3464 articles
Duration: 491.2 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-2.html
Added: 2943 articles
Duration: 410.75 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-3.html
Added: 3223 articles
Duration: 452.49 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-4.html
Added: 3074 articles
Duration: 433.31 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-5.html
Added: 3276 articles
Duration: 463.0 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-6.html
Added: 3053 articles
Duration: 434.3 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-7.html
Added: 3270 articles
Duration: 474.14 seconds
Read: https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-8.html
Added: 3370 articles
Duration: 481.03 seconds
Read: https://jdestefani.gi

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

...

Some list of limitations in text

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 Couple among employees (Question a)

Are there couples among the employees. If so, who? Are they still together?


In [2]:
import pickle
data = pickle.load(open("data/2012.pkl", "rb"))
data.head()

,0,1,2
0,Jaye Shimek,2012-01-01,15:17
1,Leandro Marse,2012-01-01,10:52
2,Annie Lapham,2012-01-02,14:36
3,Annie Lapham,2012-01-02,15:38
4,Annie Lapham,2012-01-02,09:47


In [3]:
data["datetime"] = data.iloc[:, 1] +" " +data.iloc[:, 2]

    #format to date time 
data['datetime'] = pd.to_datetime(data['datetime'],format= '%Y-%m-%d %H:%M') 
data = data.drop(data.columns[[1, 2]], axis=1)
data.columns = ['name', 'datetime']

#floor to hours
#data['datetime'] = data['datetime'].dt.floor('h')

#create multiple features to enhance analysis for Machine Learning
#data['year'] = data['datetime'].dt.year
#data['month'] = data['datetime'].dt.month
#data['day'] = data['datetime'].dt.day
#data['hour'] = data['datetime'].dt.hour
data['time'] = data['datetime'].dt.time


print(data.dtypes)
data.head()
employees = data['name'].unique().tolist()

name                object
datetime    datetime64[ns]
time                object
dtype: object


In [4]:
# group the names based on the dates and time published
df = data.set_index('datetime').groupby([pd.Grouper(freq='H'), 'name']).count()
df.rename(columns = {'time':'n'}, inplace = True)
df = df.reset_index()  # make sure indexes pair with number of rows
df

,datetime,name,n
0,2012-01-01 10:00:00,Leandro Marse,1
1,2012-01-01 15:00:00,Jaye Shimek,1
2,2012-01-02 09:00:00,Andreas Jost,1
3,2012-01-02 09:00:00,Anisha Mathes,1
4,2012-01-02 09:00:00,Annie Lapham,1
...,...,...,...
30812,2012-12-31 17:00:00,Leatrice Heer,1
30813,2012-12-31 17:00:00,Lester Preiss,1
30814,2012-12-31 17:00:00,Milagros Salmeron,1
30815,2012-12-31 17:00:00,Paulene Calcagni,1


In [5]:
publication_tsp = data['datetime'].unique()

In [7]:
# iterate to get equal publication times
l= []
for timestamp in publication_tsp:
    df_f =  df.loc[df['datetime']==timestamp]
    if df_f.shape[0] !=0:
        l.append(df_f.groupby('n')['name'].apply(list).reset_index(name='lst'))


In [68]:
print(len(l))

2088


In [8]:
from collections import Counter
from itertools import combinations

new_list = []
# create a new list of list 
for entry in l:
    lst  = entry['lst'].values.tolist()
    for items in lst:
        new_list.append(items)

# count likelihood of being a copule based on how often they occure together
d  = Counter()
for sub in new_list:
    if len(new_list) < 2:
        continue
    sub.sort()
    for comb in combinations(sub,2):
        d[comb] += 1
#create a dataframe
couples_df = pd.DataFrame(d.most_common(), columns = ['likely_couple', 'occurences'])

In [55]:
couples_df.head()

,likely_couple,occurences
0,"(Donte Berkeley, Vance Ates)",89
1,"(Donte Berkeley, Julieta Knapp)",89
2,"(Donte Berkeley, Jaye Shimek)",88
3,"(Donte Berkeley, Harmony Mcmurtry)",85
4,"(Andreas Jost, Donte Berkeley)",82


In [ ]:
# visualize data
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(vis_data['date'],vis_data['counts'])

....

In [39]:
dates = pd.date_range(start='2012-1-1', end='2012-12-31')

#extend dataframe by dates
data['date'] = pd.to_datetime(data['datetime'].dt.date)
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data

#find max time employees don't post anything
for names in employees:
    df_f =  data.loc[data['name']==names]
    print(names, df_f['date'].diff().max())



Jaye Shimek 13 days 00:00:00
Leandro Marse 12 days 00:00:00
Annie Lapham 10 days 00:00:00
Harmony Mcmurtry 17 days 00:00:00
Zofia Roll 11 days 00:00:00
Tasia Dube 14 days 00:00:00
Lorrine Paek 18 days 00:00:00
Veronique Blakemore 17 days 00:00:00
Randall Feather 14 days 00:00:00
Michale Tuch 10 days 00:00:00
Jarvis Pratts 13 days 00:00:00
Vance Ates 11 days 00:00:00
Donte Berkeley 9 days 00:00:00
Tora Echevarria 17 days 00:00:00
Hassan Toner 20 days 00:00:00
Kyle Ryals 12 days 00:00:00
Vonk Billips 17 days 00:00:00
Julieta Knapp 17 days 00:00:00
Stephine Gatlin 10 days 00:00:00
Bebe Riva 10 days 00:00:00
Marthe Hale 19 days 00:00:00
Paulene Calcagni 12 days 00:00:00
Milagros Salmeron 13 days 00:00:00
Anisha Mathes 14 days 00:00:00
Layne Woodside 13 days 00:00:00
Lisette Tolleson 12 days 00:00:00
Andreas Jost 14 days 00:00:00
Jeannie Liang 14 days 00:00:00
Hwa Defalco 13 days 00:00:00
Aurea Durrance 11 days 00:00:00
Sal Papa 14 days 00:00:00
Narcisa Core 13 days 00:00:00
Bridgette Marko

Jaye Shimek 13 days 00:00:00
Leandro Marse 12 days 00:00:00
Annie Lapham 10 days 00:00:00
Harmony Mcmurtry 17 days 00:00:00
Zofia Roll 11 days 00:00:00
Tasia Dube 14 days 00:00:00
Lorrine Paek 18 days 00:00:00
Veronique Blakemore 17 days 00:00:00
Randall Feather 14 days 00:00:00
Michale Tuch 10 days 00:00:00
Jarvis Pratts 13 days 00:00:00
Vance Ates 11 days 00:00:00
Donte Berkeley 9 days 00:00:00
Tora Echevarria 17 days 00:00:00
Hassan Toner 20 days 00:00:00
Kyle Ryals 12 days 00:00:00
Vonk Billips 17 days 00:00:00
Julieta Knapp 17 days 00:00:00
Stephine Gatlin 10 days 00:00:00
Bebe Riva 10 days 00:00:00
Marthe Hale 19 days 00:00:00
Paulene Calcagni 12 days 00:00:00
Milagros Salmeron 13 days 00:00:00
Anisha Mathes 14 days 00:00:00
Layne Woodside 13 days 00:00:00
Lisette Tolleson 12 days 00:00:00
Andreas Jost 14 days 00:00:00
Jeannie Liang 14 days 00:00:00
Hwa Defalco 13 days 00:00:00
Aurea Durrance 11 days 00:00:00
Sal Papa 14 days 00:00:00
Narcisa Core 13 days 00:00:00
Bridgette Marko

In [ ]:
ser = pd.to_datetime(pd.Series(["2012-03-01", pd.NaT]))  
ser.dt.isocalendar()  

In [41]:
# Combining all data together for analysis creating a relative file name 
pkl_Filenames = pd.date_range('2012-01-01', periods=8, freq='Y').strftime('data/%Y.pkl').tolist()
pkl_Filenames

['data/2012.pkl',
 'data/2013.pkl',
 'data/2014.pkl',
 'data/2015.pkl',
 'data/2016.pkl',
 'data/2017.pkl',
 'data/2018.pkl',
 'data/2019.pkl']

In [42]:
final_df = []
for data in pkl_Filenames:
    print(data)
    with open(data, 'rb') as f:
        if len(final_df) == 0:
            final_df = pickle.load(f)
        else:
            final_df = final_df.append(pickle.load(f))  # Update contents of file1
        #print(final_df.iloc[-1])
        f.close()

final_df.columns =['name', 'date', 'time'] # name the columns
final_df['date'] = pd.to_datetime(final_df['date'],format= '%Y-%m-%d') 
final_df['time'] = pd.to_datetime(final_df['time'],format= '%H:%M' ).dt.time

Filename = 'data/2012_2019.pkl' 
with open(Filename, 'wb') as handle:
    pickle.dump(final_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

merged = pickle.load(open(Filename, "rb"))



data/2012.pkl
data/2013.pkl
data/2014.pkl
data/2015.pkl
data/2016.pkl
data/2017.pkl
data/2018.pkl
data/2019.pkl


/var/folders/qd/wtyzdplx1712rf3p12f4blkm0000gn/T/ipykernel_58645/2869434471.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(pickle.load(f))  # Update contents of file1
/var/folders/qd/wtyzdplx1712rf3p12f4blkm0000gn/T/ipykernel_58645/2869434471.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(pickle.load(f))  # Update contents of file1
/var/folders/qd/wtyzdplx1712rf3p12f4blkm0000gn/T/ipykernel_58645/2869434471.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(pickle.load(f))  # Update contents of file1
/var/folders/qd/wtyzdplx1712rf3p12f4blkm0000gn/T/ipykernel_58645/2869434471.py:8: FutureWarning: The frame.append method is deprecated a

In [43]:
merged.head()

,name,date,time
0,Jaye Shimek,2012-01-01,15:17:00
1,Leandro Marse,2012-01-01,10:52:00
2,Annie Lapham,2012-01-02,14:36:00
3,Annie Lapham,2012-01-02,15:38:00
4,Annie Lapham,2012-01-02,09:47:00


### 2.1.1 Check month of interest 

Description of results

## 2.2 Children (Question b)
Did any of the employees have a child? If so, who?

In [2]:
#Code for question b

Description of results

## 2.3 Holidays (Question c)
If you would be looking to work for Tabularazor Inc., how many holidays can you expect to get per
year?


In [ ]:
#Code for question c

Description of results

# 3 Conclusion

Add code if needed